In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from PIL import Image,ImageDraw
from PIL import ImageFont

import tensorflow as tf # TF2


def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]


if __name__ == '__main__':
    face_model = "./face_landmark.tflite"
    iris_model = "./iris_landmark.tflite"
    landmark_file = "./landmark_contours.txt"

    interpreter = tf.lite.Interpreter(model_path=face_model)
    interpreter.allocate_tensors()

    iris_interpreter = tf.lite.Interpreter(model_path=iris_model)
    iris_interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    print("face landmark input info:",input_details)
    output_details = interpreter.get_output_details()

    iris_input = iris_interpreter.get_input_details()
    print("iris landmark input info:",iris_input)
    iris_output = iris_interpreter.get_output_details()

    # check the type of the input tensor
    floating_model = input_details[0]['dtype'] == np.float32
    iris_float_model = iris_input[0]["dtype"] == np.float32

    # NxHxWxC, H:1, W:2
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    iris_height = iris_input[0]["shape"][1]
    iris_width = iris_input[0]["shape"][2]

    print("height = ",height,"width = ",width)
    img = Image.open("./img/gst_zero_33.png").resize((width, height))
    draw = ImageDraw.Draw(img)
    print("numpy img size = ",img.size)

    eye_points = open(landmark_file,"r")
    left_eye_points = []
    right_eye_points = []
    lips_points = []
    for line in eye_points:
        points = line.split(" ")
        if points[0] == "left_eye":
            for data in points[1:-1]:
                left_eye_points.append(int(data))
        elif points[0] == "right_eye":
            for data in points[1:-1]:
                right_eye_points.append(int(data))
        elif points[0] == "lips":
            for data in points[1:-1]:
                lips_points.append(int(data))

    print("left eye points ",left_eye_points)
    print("right eye points ",right_eye_points)

    image = img
    if(image.mode == str('RGBA')):
        print("rgba show\n")
        r, g, b, a = image.split()
        img = Image.merge("RGB", (r, g, b))
        image = img

    image.show("rgb show")
    draw = ImageDraw.Draw(image)


    # add N dim
    input_data = np.expand_dims(img, axis=0)

    if floating_model:
        input_data = (np.float32(input_data) - 128) / 256 

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    results = np.squeeze(output_data)

    output_data1= interpreter.get_tensor(output_details[1]["index"])
    print("output data 1 = ",output_data1)

    print ("result size = ",len(results))
    output_file = open("point_file.txt","w")
    point_list = []
    color_list = [0,0,0]
    left_eye_list = [[],[]]
    right_eye_list = [[],[]]
    left_eye_center = [0,0]
    right_eye_center = [0,0]
    for index in range(int(len(results)/3)):
        point = (results[3*index],results[3*index + 1])
        if index in left_eye_points:
            draw.point(point,(255,0,0))
            # draw.text([point[0],point[1]],str(index),fill=(100,185,179),)
            #print(index,point,"\n")
            left_eye_list[0].append(point[0])
            left_eye_list[1].append(point[1])
        elif index in right_eye_points:
            draw.point(point,(255,0,0))
            #print("right:",index,point,"\n")
            right_eye_list[0].append(point[0])
            right_eye_list[1].append(point[1])
        elif index in lips_points:
            draw.point(point,(255,0,0))
            print("lips:",index,point,"\n")
            right_eye_list[0].append(point[0])
            right_eye_list[1].append(point[1])
        else :
            draw.point(point,(0,255,0))
        # draw.point(point,tuple( color_list))
        point_list.append(point)
        output_file.writelines(str(point))
        output_file.write("\n")

    left_v_point_bot = [(results[3*145]+results[3*153])/2,(results[3*145+1]+results[3*153+1])/2]
    left_v_point_top = [(results[3*159]+results[3*158])/2,(results[3*159+1]+results[3*158+1])/2]

    left_v_point_left = [results[3*33],results[3*33+1]]
    left_v_point_right = [results[3*133],results[3*133+1]]

    EAR = ((left_v_point_bot[1]) - (left_v_point_top[1]) )/(left_v_point_right[0]-left_v_point_left[0])
    print("EAR value is ",EAR)


    lips_distance_h = results[3*409] - results[3*185]
    lips_distance_v1 = results[3*84 +1] - results[3*37 +1]
    lips_distance_v2 = results[3*17 +1] - results[3*0 +1]
    lips_distance_v3 = results[3*314 +1] - results[3*267 +1]

    lips_ear = (lips_distance_v1 + lips_distance_v2 + lips_distance_v3) / (3 * lips_distance_h)

    print("LIPS EAR value is ",lips_ear)

    print("left eye center count",left_eye_list)
    left_eye_center[1] = sum(left_eye_list[1])/len(left_eye_list[1])
    left_eye_center[0] = sum(left_eye_list[0])/len(left_eye_list[0])

    right_eye_center[1] = sum(right_eye_list[1])/len(right_eye_list[1])
    right_eye_center[0] = sum(right_eye_list[0])/len(right_eye_list[0])
    print("left eye center",left_eye_center)
    draw.point(left_eye_center,(0,0,255))
    draw.point(right_eye_center,(0,0,255))
    # draw.point((71.96,91.39),(0,0,255))
    # draw.point(point_list,(0,255,0))

    left_eye_box = (left_eye_center[0]-32,left_eye_center[1]-32,left_eye_center[0]+32,left_eye_center[1]+32)
    right_eye_box = (right_eye_center[0]-32,right_eye_center[1]-32,right_eye_center[0]+32,right_eye_center[1]+32)
    iris_left_img = Image.open("./img/face.jpeg").resize((width, height)).crop(left_eye_box)
    iris_right_img = Image.open("./img/face.jpeg").resize((width, height)).crop(right_eye_box)
    img.show("sources img")

    # add N dim
    iris_input_data = np.expand_dims(iris_left_img, axis=0)

    if iris_float_model:
        iris_input_data = (np.float32(iris_input_data) - 128) / 256 

    iris_interpreter.set_tensor(iris_input[0]['index'], iris_input_data)

    iris_interpreter.invoke()

    iris_output_data = iris_interpreter.get_tensor(iris_output[0]['index'])
    iris_results = np.squeeze(iris_output_data)

    iris_output_data1= iris_interpreter.get_tensor(iris_output[1]["index"])
    print("iris data 1 = ",iris_output_data1)
    iris_results = np.squeeze(iris_output_data1)
    
    iris_draw = ImageDraw.Draw(iris_left_img)
    for index in range(int(len(iris_results)/3)):
        point = (iris_results[3*index],iris_results[3*index + 1])
        iris_draw.point(point,(0,0,255))
        iris_draw.text([point[0],point[1]],str(index),fill=(100,185,179))
    iris_left_img.show("left eye img")
    



face landmark input info: [{'name': 'input_1', 'index': 0, 'shape': array([  1, 192, 192,   3], dtype=int32), 'shape_signature': array([  1, 192, 192,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
iris landmark input info: [{'name': 'input_1', 'index': 0, 'shape': array([ 1, 64, 64,  3], dtype=int32), 'shape_signature': array([ 1, 64, 64,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
height =  192 width =  192
numpy img size =  (192, 192)
left eye points  [33, 7, 7, 163, 163, 144, 144, 145, 145, 153, 153, 154, 154, 155, 155, 133, 33, 246, 246, 161, 161, 160, 160, 159, 159, 158, 158, 157, 157, 173, 173]
right e

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from PIL import Image,ImageDraw
from PIL import ImageFont

import tensorflow as tf # TF2


def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]

SSD_OPTIONS_FRONT = {
    'num_layers': 4,
    'input_size_height': 128,
    'input_size_width': 128,
    'anchor_offset_x': 0.5,
    'anchor_offset_y': 0.5,
    'strides': [8, 16, 16, 16],
}

def _ssd_generate_anchors(opts: dict) -> np.ndarray:
    """This is a trimmed down version of the C++ code; all irrelevant parts
    have been removed.
    (reference: mediapipe/calculators/tflite/ssd_anchors_calculator.cc)
    """
    layer_id = 0
    num_layers = opts['num_layers']
    strides = opts['strides']
    assert len(strides) == num_layers
    input_height = opts['input_size_height']
    input_width = opts['input_size_width']
    anchor_offset_x = opts['anchor_offset_x']
    anchor_offset_y = opts['anchor_offset_y']
    anchors = []
    while layer_id < num_layers:
        last_same_stride_layer = layer_id
        repeats = 0
        while (last_same_stride_layer < num_layers and
               strides[last_same_stride_layer] == strides[layer_id]):
            last_same_stride_layer += 1
            repeats += 2    # aspect_ratios are added twice per iteration
        stride = strides[layer_id]
        feature_map_height = input_height // stride
        feature_map_width = input_width // stride
        for y in range(feature_map_height):
            y_center = (y + anchor_offset_y) / feature_map_height
            for x in range(feature_map_width):
                x_center = (x + anchor_offset_x) / feature_map_width
                for _ in range(repeats):
                    anchors.append((x_center, y_center))
        layer_id = last_same_stride_layer
    return np.array(anchors, dtype=np.float32)


if __name__ == '__main__':
    detection_model = "./face_detection_front.tflite"

    interpreter = tf.lite.Interpreter(model_path=detection_model)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    print("face landmark input info:",input_details)
    output_details = interpreter.get_output_details()

    # check the type of the input tensor
    floating_model = input_details[0]['dtype'] == np.float32

    # NxHxWxC, H:1, W:2
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    print("height = ",height,"width = ",width)
    image = Image.open("./img/face.jpeg").resize((width, height))
    print(image.mode)
    print(np.int32(image.getdata()))
    if(image.mode == str('RGBA')):
      r, g, b, a = image.split()
      image = Image.merge("RGB", (r, g, b))

    draw = ImageDraw.Draw(image)
    print("numpy img size = ",image.size)

    # add N dim
    input_data = np.expand_dims(image, axis=0)

    if floating_model:
      print("float\n")
      input_data = (np.float32(input_data) -128) / 256

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    results = np.squeeze(output_data)

    output_data1= interpreter.get_tensor(output_details[1]["index"])

    # resutl_file = open("./face_detection.txt","w")
    # index = 0
    # for point in output_data:
    #   if(output_data1[0][index][0] > -10) :
    #     resutl_file.writelines(str(point))
    #     resutl_file.write("\n")
    #   index += 1
    # width == height so scale is the same across the board
    scale = interpreter.get_input_details()[0]['shape'][1]
    print("scale value == ",scale,"\n")
    num_points = output_data.shape[-1] // 2
    # scale all values (applies to positions, width, and height alike)
    boxes = output_data.reshape(-1, num_points, 2) / scale
    # adjust center coordinates and key points to anchor positions
    boxes[:, 0] += _ssd_generate_anchors(SSD_OPTIONS_FRONT)
    for i in range(2, num_points):
        boxes[:, i] += _ssd_generate_anchors(SSD_OPTIONS_FRONT)
    # convert x_center, y_center, w, h to xmin, ymin, xmax, ymax
    center = np.array(boxes[:, 0])
    half_size = boxes[:, 1] / 2
    boxes[:, 0] = center - half_size
    boxes[:, 1] = center + half_size

    print("boxes value :\n",boxes)

    first_point = output_data[0][0]
    print(first_point)
    print("scores value :\n",output_data1)
    draw.rectangle((first_point[0]*128 -  int(first_point[2]/2),first_point[1]*128 -  int(first_point[3]/2),first_point[0]*128 + int(first_point[2]/2),first_point[1]*128 + int(first_point[3]/2)))
    image.show("face detection")




In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import cv2

import tensorflow as tf # TF2


if __name__ == '__main__':
  detection_model = "./traffic_sign_detect.tflite"

  interpreter = tf.lite.Interpreter(model_path=detection_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()
  print("face landmark input info:",input_details)
  output_details = interpreter.get_output_details()
  print("face landmark output info:",output_details)

  # check the type of the input tensor
  floating_model = input_details[0]['dtype'] == np.float32

  # NxHxWxC, H:1, W:2
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  print("height = ",height,"width = ",width)
  img = cv2.imread("./img/00633.png")
  image = cv2.resize(img,(width,height))

  # add N dim
  input_data = np.expand_dims(image, axis=0)

  if floating_model:
    print("float\n")
    input_data = (np.float32(input_data) -128) / 256

  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()

  boxes = interpreter.get_tensor(output_details[0]['index'])
  results = np.squeeze(boxes)

  classes = interpreter.get_tensor(output_details[1]['index'])

  scores = interpreter.get_tensor(output_details[2]['index'])

  cv2.imshow("source",img)
  print("boxes value:",boxes)
  print("classes value:",classes)
  print("scores value:",scores)

  i = 0
  score_arr = scores[0]
  for score in score_arr:
    if score > 0.6 :
      cv2.rectangle(img,(boxes[i][0] * width,boxes[i][1] * height ),(boxes[i][2] * width,boxes[i][3] * width),(255,0,0))
    i += 1


face landmark input info: [{'name': 'normalized_input_image_tensor', 'index': 412, 'shape': array([  1, 320, 320,   3], dtype=int32), 'shape_signature': array([  1, 320, 320,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
face landmark output info: [{'name': 'TFLite_Detection_PostProcess', 'index': 404, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'TFLite_Detection_PostProcess:1', 'index': 405, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantizati